# Serverless cost curves

This notebook calculates costs for some EC2 and Lambda flavors by taking a uniform distribution of requests during a whole month


## 0. Initial setup

### Imports

In [ ]:
from bbva_colors import BBVAcolors
import awscosts
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
SECONDS_IN_A_MONTH = 3600 * 24 * 30

### User variables

Modify this to play with different values

In [ ]:
ec2_flavors = ('m3.medium', 'm4.large', 'm4.4xlarge')
reqs_range = range(1, 10000, 10)

lambda_memory = 128 # MiB
lambda_time = 200 # ms

image_type = None
# Uncomment next line to produce pngs
# image_type = 'png'

## 1. Generate and plot costs for uniform distribution of requests

### 1.1 Request generation and costs calculation

In [ ]:
costs = list()

# for throughput_ratio in [1, 5, 10]:
throughput_ratio = 1
cost = dict()
cost['throughput_ratio'] = throughput_ratio
cost['data'] = list()
cost=dict()

for flavor in ec2_flavors:
    myec2 = awscosts.EC2(
        flavor,
        MB_per_req=lambda_memory,
        ms_per_req=lambda_time,
        throughput_ratio=throughput_ratio,
    )
    cost[flavor]=dict()
    for reqs_per_second in reqs_range:
        cost[flavor][reqs_per_second] = myec2.get_cost_per_second(reqs_per_second) * SECONDS_IN_A_MONTH

mylambda = awscosts.Lambda(
    MB_per_req=lambda_memory,
    ms_per_req=lambda_time,
)
cost['lambda']=dict()
for reqs_per_second in reqs_range:
    requests_per_month = reqs_per_second * SECONDS_IN_A_MONTH
    cost['lambda'][reqs_per_second] = mylambda.get_cost(requests_per_month, reset_free_tier=True)



### 1.2 Plot

In [ ]:
title = 'Monthy cost by number of reqs/s'
data = []

color = iter(
    [BBVAcolors['light'], BBVAcolors['aqua'], BBVAcolors['navy'], BBVAcolors['coral']]
)

for flavor in cost.keys():
    x = list()
    y = list()
    for reqs, price in cost[flavor].items():
        x.append(reqs)
        y.append(price)

    trace = go.Scatter(
        x=x,
        y=y,
        name=flavor,
        marker=next(color)
    )
    data.append(trace)


layout = go.Layout(
    title=title,
    legend=dict(
        orientation="h",
        y=-.2,
    ),
    width=1000,
    height=800,
    xaxis=dict(
        title='<b>reqs/sec</b>',
        type='log'
    ),
    yaxis=dict(
        title='<b>Monthly cost ($)</b>',
        type='log'
    )
)

figure = go.Figure(data=data, layout=layout)

iplot(figure, image=image_type)